In [ ]:
import pandas as pd
# Imports from original code (keep them in case needed elsewhere)
import os
import csv
from pathlib import Path
import datetime as dt

# --- 1. Load Data ---
print("Loading data...")
zipcodes_coords_df = pd.read_csv("./resources/Zipcode.csv")
time_df = pd.read_csv("./resources/Sale_Price.csv")
cpi_df_raw = pd.read_csv("./resources/Inflation.csv")
interest_df_raw = pd.read_csv("./resources/Interest_Rates.csv")
mortgage_df_raw = pd.read_csv("./resources/Mortgage_data.csv")
unemployment_df_raw = pd.read_csv("./resources/Unemployment_Rate.csv")
print("Data loading complete.")

# --- 2. Preprocess Time Data (Filtering and Transposing) ---
print("Processing time data...")
# Get list of valid RegionIDs
good_id_list = zipcodes_coords_df['RegionID'].tolist()

# Filter time_df efficiently using isin()
# Ensure RegionID types match if necessary before filtering (assuming they are compatible here)
time_df_filtered = time_df[time_df['RegionID'].isin(good_id_list)].copy() # Use .copy() to avoid SettingWithCopyWarning

# Prepare for transpose: Drop unnecessary columns, set RegionID as index
time_df_processed = time_df_filtered.drop(columns=["RegionName", "StateName", "SizeRank"])
time_df_processed = time_df_processed.astype({'RegionID': 'str'}) # Ensure RegionIDs are strings for column names
time_df_processed = time_df_processed.set_index('RegionID')

# Transpose: Dates become index, RegionIDs become columns
time_transposed = time_df_processed.transpose()

# Clean up the transposed data: Convert index (dates) to datetime, name it, reset to column
time_transposed.index.name = 'DATE'
time_transposed.index = pd.to_datetime(time_transposed.index, infer_datetime_format=True)
time_transposed = time_transposed.reset_index()
print("Time data processing complete.")

# --- 3. Preprocess Supplementary Economic Data ---
print("Processing economic data...")
# CPI Data
cpi_df = cpi_df_raw[['DATE', 'CPIAUCNS']].copy()
cpi_df['DATE'] = pd.to_datetime(cpi_df['DATE'], infer_datetime_format=True)
cpi_df = cpi_df.rename(columns={'CPIAUCNS': 'INFLATION'})

# Interest Rate Data
interest_df = interest_df_raw[['DATE', 'FEDFUNDS']].copy()
interest_df['DATE'] = pd.to_datetime(interest_df['DATE'], infer_datetime_format=True)
interest_df = interest_df.rename(columns={'FEDFUNDS': 'INTEREST_RATES'})

# Mortgage Rate Data
mortgage_df = mortgage_df_raw.copy()
# Rename columns robustly - assuming rate is the second column
mortgage_rate_col_name = mortgage_df.columns[1] # Get actual name like 'Average\r\nContract\r\nRate'
mortgage_df = mortgage_df.rename(columns={'Month': 'DATE', mortgage_rate_col_name: 'AVG_MORTGAGE_RATE'})
mortgage_df['DATE'] = pd.to_datetime(mortgage_df['DATE'], infer_datetime_format=True)
mortgage_df = mortgage_df[['DATE', 'AVG_MORTGAGE_RATE']] # Select only needed columns

# Unemployment Rate Data
unemployment_df = unemployment_df_raw[['TIME', 'Value']].copy()
unemployment_df['DATE'] = pd.to_datetime(unemployment_df['TIME'], infer_datetime_format=True)
unemployment_df = unemployment_df.rename(columns={'Value': 'UNEMPLOYMENT_RATE'})
unemployment_df = unemployment_df[['DATE', 'UNEMPLOYMENT_RATE']] # Select and drop old TIME column
print("Economic data processing complete.")

# --- 4. Merge DataFrames ---
print("Merging data...")
# Start with the transposed time series data
merged_df = time_transposed.copy()

# List of dataframes to merge
dfs_to_merge = [cpi_df, interest_df, mortgage_df, unemployment_df]

# Sequentially merge using 'inner' join on 'DATE'
for df in dfs_to_merge:
    merged_df = pd.merge(merged_df, df, on='DATE', how='inner')

print("Data merging complete.")

# --- 5. Final Cleaning and Output ---
print("Finalizing data...")
# Drop rows with any NaN values resulting from merges or original data
# Consider alternative NaN strategies (imputation) if dropping loses too much data
final_df = merged_df.dropna()

# Optional: Reorder columns to place economic indicators first
price_columns = [col for col in final_df.columns if col not in ['DATE', 'INFLATION', 'INTEREST_RATES', 'AVG_MORTGAGE_RATE', 'UNEMPLOYMENT_RATE']]
column_order = ['DATE', 'INFLATION', 'INTEREST_RATES', 'AVG_MORTGAGE_RATE', 'UNEMPLOYMENT_RATE'] + price_columns
final_df = final_df[column_order]

print("--- Final Merged DataFrame ---")
print(final_df.info())
print(final_df.head())

# Example: Save the final dataset (uncomment to run)
output_path = 'full_data_set_merged.csv'
final_df.to_csv(output_path, index=False)
print(f"Final dataset saved to {output_path}")